# ZenML MLOps Template - Tutorial

This notebook demonstrates how to use the ZenML MLOps template with all available make commands.

## Prerequisites

- Docker and Docker Compose installed
- Make installed
- Run this notebook from the project root directory

## 1. View Available Commands

First, let's see all available make commands:

In [ ]:
!make help

## 2. Start All Services

Start the infrastructure (ZenML, MLflow, MySQL, Inference API, Prometheus, Grafana):

In [ ]:
!make up

Wait for services to be ready (especially MySQL and ZenML):

In [ ]:
import time
print("Waiting 60 seconds for services to initialize...")
time.sleep(60)
print("Done!")

Check that all services are running:

In [ ]:
!docker compose ps

## 3. Run the Training Pipeline

Run the ML training pipeline. On first run, this will:
1. Activate the ZenML server (create admin user)
2. Create a service account for authentication
3. Generate an API key
4. Execute the preprocessing and training steps

In [ ]:
!make train

## 4. Check API Health

Verify the inference API is running and has loaded the model:

In [ ]:
!make health

## 5. Make a Prediction

Send a prediction request to the inference API:

In [ ]:
!make predict

You can also make predictions programmatically:

In [ ]:
import requests
import json

# Sample Iris flower measurements
data = {
    "sepal_length": 5.1,
    "sepal_width": 3.5,
    "petal_length": 1.4,
    "petal_width": 0.2
}

response = requests.post(
    "http://localhost:8000/predict",
    json=data
)

print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

# Map prediction to class name
classes = ["setosa", "versicolor", "virginica"]
prediction = response.json().get("prediction", 0)
print(f"\nPredicted class: {classes[prediction]}")

## 6. Trigger Model Retraining

Trigger a model retraining via the API endpoint:

In [ ]:
!make retrain

## 7. View Service Logs

View logs from all services (press Ctrl+C to stop):

In [ ]:
# View last 20 lines of logs from each service
!docker compose logs --tail=20

## 8. Access the Dashboards

The following dashboards are available:

| Service | URL | Credentials |
|---------|-----|-------------|
| ZenML Dashboard | http://localhost:8888 | admin / zenml |
| MLflow UI | http://localhost:5001 | - |
| Inference API Docs | http://localhost:8000/docs | - |
| Prometheus | http://localhost:9092 | - |
| Grafana | http://localhost:3002 | admin / admin |

In [ ]:
# Open dashboards in browser (uncomment to use)
# import webbrowser
# webbrowser.open("http://localhost:8888")  # ZenML
# webbrowser.open("http://localhost:5001")  # MLflow
# webbrowser.open("http://localhost:8000/docs")  # API Docs

## 9. Check Prometheus Metrics

View the metrics exposed by the inference API:

In [ ]:
!curl -s http://localhost:8000/metrics | head -30

## 10. Build Docker Images

Rebuild all Docker images (useful after code changes):

In [ ]:
!make build

## 11. Stop All Services

Stop all running containers:

In [ ]:
!make down

## 12. Full Cleanup

Remove all containers, volumes, and data (for a fresh start):

In [ ]:
# WARNING: This will delete all data!
# Uncomment to run:
# !make clean

## Summary

### Quick Reference

| Command | Description |
|---------|-------------|
| `make help` | Show all available commands |
| `make up` | Start all services |
| `make down` | Stop all services |
| `make build` | Build Docker images |
| `make logs` | View service logs |
| `make train` | Run training pipeline |
| `make retrain` | Trigger retraining via API |
| `make predict` | Make example prediction |
| `make health` | Check API health |
| `make clean` | Remove all data |

### Architecture

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Pipeline       │────▶│  ZenML Server   │────▶│     MySQL       │
│  Runner         │     │  (Port 8888)    │     │  (Port 3306)    │
└─────────────────┘     └─────────────────┘     └─────────────────┘
        │
        ▼
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│     MLflow      │◀────│  Inference API  │────▶│   Prometheus    │
│  (Port 5001)    │     │  (Port 8000)    │     │  (Port 9092)    │
└─────────────────┘     └─────────────────┘     └─────────────────┘
                                                        │
                                                        ▼
                                                ┌─────────────────┐
                                                │    Grafana      │
                                                │  (Port 3002)    │
                                                └─────────────────┘
```